# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846820900487                   -0.70    3.8
  2   -7.852300689881       -2.26       -1.53    1.0
  3   -7.852646080004       -3.46       -2.53    3.2
  4   -7.852646678111       -6.22       -3.35    2.5
  5   -7.852646685987       -8.10       -4.81    1.0
  6   -7.852646686727       -9.13       -5.24    4.2
  7   -7.852646686730      -11.51       -5.99    2.0
  8   -7.852646686730      -12.72       -7.49    2.0
  9   -7.852646686730   +  -15.05       -7.90    3.0
 10   -7.852646686730   +  -14.75       -9.23    2.0
 11   -7.852646686730      -14.35       -9.93    2.2
 12   -7.852646686730   +  -15.05       -9.28    1.0
 13   -7.852646686730   +  -15.05       -9.19    1.0
 14   -7.852646686730   +  -14.75       -8.63    1.8
 15   -7.852646686730      -14.57       -9.63    2.0
 16   -7.852646686730   +  -14.75      -10.04    1.8
 17   -7.852646686730      -14.75      -10.68 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846870529455                   -0.70           4.0
  2   -7.852526096886       -2.25       -1.62   0.80    5.0
  3   -7.852609907709       -4.08       -2.79   0.80    5.2
  4   -7.852646570582       -4.44       -3.57   0.80    7.8
  5   -7.852646683333       -6.95       -4.50   0.80    8.8
  6   -7.852646686665       -8.48       -5.06   0.80   10.5
  7   -7.852646686724      -10.23       -6.08   0.80   11.2
  8   -7.852646686730      -11.26       -7.19   0.80   13.0
  9   -7.852646686730      -13.30       -7.76   0.80   16.0
 10   -7.852646686730      -14.45       -8.59   0.80   17.2
 11   -7.852646686730   +  -14.57       -9.57   0.80   18.2
 12   -7.852646686730   +  -14.75      -10.28   0.80   19.5
 13   -7.852646686730      -15.05      -11.17   0.80   20.8
 14   -7.852646686730      -15.05      -11.69   0.80   22.0
 15   -7.852646686730      -14.75      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.361037e+01     3.501077e+00
 * time: 0.48682188987731934
     1     1.249653e+00     2.049905e+00
 * time: 0.8184490203857422
     2    -1.574991e+00     2.436599e+00
 * time: 0.8462738990783691
     3    -3.711726e+00     2.113564e+00
 * time: 0.8865599632263184
     4    -5.067672e+00     1.755272e+00
 * time: 0.9267361164093018
     5    -6.810763e+00     9.878781e-01
 * time: 0.9678659439086914
     6    -6.824058e+00     1.159208e+00
 * time: 0.9967260360717773
     7    -7.501957e+00     7.181530e-01
 * time: 1.0255210399627686
     8    -7.561702e+00     7.648032e-01
 * time: 1.0535659790039062
     9    -7.718395e+00     5.232774e-01
 * time: 1.0815339088439941
    10    -7.777781e+00     3.801229e-01
 * time: 1.1097888946533203
    11    -7.832661e+00     6.986044e-02
 * time: 1.1377780437469482
    12    -7.843578e+00     4.607336e-02
 * time: 1.165532112121582
    13    -7.850309e+00     2.751674e-02
 * time: 1.1932520866

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846813545927                   -0.70    4.2
  2   -7.852318535933       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686702                   -2.55
  2   -7.852646686730      -10.55       -5.85
  3   -7.852646686730   +  -14.75      -12.54
|ρ_newton - ρ_scf|  = 1.9568480204171013e-13
|ρ_newton - ρ_scfv| = 6.929836761582625e-14
|ρ_newton - ρ_dm|   = 2.186256858575344e-9
